In [4]:
from time import time
import numpy as np
import copy
import collections


def AC3(csp, neighbor):
    """
    :param csp: dict{X: list of variables,
                     D: dictionary {variable: [domain]},
                     C: dictionary {(node_i, node_j): list of constraints}
    :param neighbor: dict {var: [neighbor]}
    :return: a (bool, arc consistent domain(if exist)) with false if an inconsistency is found and true otherwise
    """
    queue = []
    for x in neighbor.keys():
        if len(neighbor[x]) > 0:
            for y in neighbor[x]:
                queue.append((x, y))

    counts = []
    count_for = 0
    while len(queue) != 0:
        arc = queue.pop(0)
        res, csp_new, c = revise(csp, arc[0], arc[1])
        csp = csp_new
        if res:
            if len(csp["D"][arc[0]]) == 0:
                counts.append(c)
                print("AC3 require ", len(counts), " while steps. In average each of those require ", np.average(counts), " steps. The total of basic operations is ", np.sum(counts)) #len(counts)*np.sum(counts)+count_for)
                return False, csp["D"]
            for Xk in neighbor[arc[0]]:
                count_for += 1
                if Xk != arc[1]:
                    queue.append((Xk, arc[0]))
        counts.append(max(c,count_for))
    print("AC3 require ", len(counts), " while steps. In average each of those require ", np.average(counts), " steps. The total of basic operations is ", np.sum(counts))#len(counts)*np.sum(counts))+count_for)
    return True, csp["D"]


def revise(csp, Xi, Xj):
    """
    This function try to revise the domain
    :param csp: list(X,D,C) where X is a list of variables, D is a dictionary {variable: [domain]},
                and C is a list of constraints
    :param Xi: node i
    :param Xj: node j
    :return: a (bool, csp) with true iff we revise the domain of Xi
    """
    count = 0
    revised = False
    if (Xi, Xj) in csp["C"].keys():
        for Vi in csp["D"][Xi]:
            satisfy = False
            for Vj in csp["D"][Xj]:
                count += 1
                if (Vi, Vj) in csp["C"][(Xi, Xj)]:
                    satisfy = True
                    break
            if not satisfy:
                csp["D"][Xi].remove(Vi)
                revised = True

    return revised, csp, count

if __name__ == "__main__":

    """
    read the file and change it into Graph Structure
    """
    e = []
    f = open('smalldata.txt', 'r')    # my e.txt is a little different from the txt given by Professor Arora. I addad a '#' in the 3th line 'colors = 4 '
    for lines in f:
        ls = lines.strip('\n').replace(' ', ',').split(",")
        if ls[0] == "#":
            continue
        e.append(ls)
    f.close()

    graph = collections.defaultdict(list)
    X = []
    for line in e:
        graph[str(line[0])].append(str(line[1]))
        graph[str(line[1])].append(str(line[0]))
        if str(line[0]) not in X:X.append(str(line[0]))
        if str(line[1]) not in X:X.append(str(line[1]))

    """
    param csp: dict{X: list of variables,
                    D: dictionary {variable: [domain]},
                    C: dictionary {(node_i, node_j): list of constraints}   
    """

    D = {}
    for el in X:
        if el == '1':
            D[el] = ["R"]
        else:
            D[el] = ["R", "G", "B", "Y"]
    constr = [(x, y) for x in ["R", "G", "B", "Y"] for y in ["R", "G", "B", "Y"] if x != y]
    C = {}
    for node in graph.keys():
        for neighbor in graph[node]:
            C[(node, neighbor)] = constr

    # Run AC3
    start = time()
    consistency, value = AC3(copy.deepcopy({"X": X, "D": D, "C": C}), graph)
    s = time() - start
    if consistency:
        print("AC3 -> Exist consistent solution: ", value)
    else:
        print("AC3 -> No consistent solution!")
    print("solution retrieved in: %.4g s" % s)
    print("\n")






AC3 require  50  while steps. In average each of those require  15.56  steps. The total of basic operations is  778
AC3 -> Exist consistent solution:  {'1': ['R'], '2': ['G', 'B', 'Y'], '3': ['G', 'B', 'Y'], '4': ['G', 'B', 'Y'], '5': ['G', 'B', 'Y'], '6': ['R', 'G', 'B', 'Y'], '7': ['R', 'G', 'B', 'Y']}
solution retrieved in: 0.0007951 s


